# Bayesian linear regression model [optional]

In this notebook we fit a [Bayesian linear regression](https://en.wikipedia.org/wiki/Bayesian_linear_regression) model to the data. 
This serves mainly as a useful baseline and tells us if there is a strong linear relationship between the inputs and output. 
If the linear model fits the data well, then perhaps there is no reason to apply a more complicated model!

The advantage of using a Bayesian approach over a classical linear regression model in this case is that the Bayesian inference provides us with distributions of all parameters instead of just point estimates, which allows us to perform some additional analyses.

Note that this step is optional and not strictly necessary to complete the rest of the tutorial. 

## Dependencies

First we import the required dependencies.

If you are in Colab, you need to install the [pyro](https://pyro.ai/) package by uncommenting and running the line `!pip3 install pyro-ppl` below before proceeding.

In [ ]:
# install dependencies
# !pip3 install pyro-ppl

In [ ]:
# imports
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import pyro

pyro.set_rng_seed(0)
print(f"torch version: {torch.__version__}")
print(f"pyro version: {pyro.__version__}")

## Load dataset

We can load the dataset directly from the GitHub URL.
Alternatively, the dataset can be loaded from a local file.

In [ ]:
# load dataset
dataset_path = "https://raw.githubusercontent.com/BIG-MAP/sensitivity_analysis_tutorial/main/data/p2d_sei_10k.csv"
# dataset_path = "data/p2d_sei_10k.csv"  # local
df = pd.read_csv(dataset_path, index_col=0)

# store the names of the features and the name of the target variable
features = df.columns[:15].tolist()  # use input parameters as features
target = "SEI_thickness(m)"  # primary target
# target = "Capacity loss (%)"  # secondary target

## Prepare training and validation data

In preparation for training the machine learning model we do a few data transformations:

* The target variable is log transformed and normalised to zero mean and unit variance.
* The input features are normalised to zero mean and unit variance to make the model parameters easier to learn and to put the inputs on the same scale and thus make results for each input directly comparable. 

Finally, the data is split into a training and a validation set. 

In [ ]:
# helper functions

def create_data_split_index(n_data, n_train, n_valid=None, shuffle=False):
    """Create data split index."""
    n_valid = n_data - n_train if n_valid is None else n_valid        
    index = torch.randperm(n_data) if shuffle else torch.arange(n_data)
    split = {
        "train": index[:n_train],
        "valid": index[n_train:n_train + n_valid],
        "rest":  index[n_train + n_valid:],
    }
    return split

def create_normaliser(x, y):
    """Create data normalisation function"""
    x_mean, x_std = x.mean(axis=0), x.std(axis=0)
    y_mean, y_std = y.mean(axis=0), y.std(axis=0)
    def normaliser(x, y):
        return (x - x_mean) / x_std, (y - y_mean) / y_std
    normaliser_params = {"x_mean": x_mean, "x_std": x_std, "y_mean": y_mean, "y_std": y_std}
    return normaliser, normaliser_params

In [ ]:
# settings
shuffle = False
n_data = len(df)
n_train = 5000
n_valid = 5000

assert n_train + n_valid <= n_data

# create data tensors
x_data_orig = torch.tensor(df[features].values, dtype=torch.float)
y_data_orig = torch.tensor(df[target].values, dtype=torch.float)

# log transform y
y_data_orig = torch.log(y_data_orig)

# create data split index
split = create_data_split_index(n_data, n_train, n_valid)

# create normalisation function from training split
normaliser, normaliser_params = create_normaliser(x_data_orig[split["train"]], y_data_orig[split["train"]])

# normalize data
x_data, y_data = normaliser(x_data_orig, y_data_orig)

# create data splits 
x_train, y_train = x_data[split["train"]], y_data[split["train"]]
x_valid, y_valid = x_data[split["valid"]], y_data[split["valid"]]

assert len(x_train) == len(y_train) == n_train
assert len(x_valid) == len(y_valid) == n_valid

n_bins = 50
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.hist(y_train.numpy(), bins=n_bins)
plt.xlabel("y_train")
plt.subplot(122)
plt.hist(y_valid.numpy(), bins=n_bins)
plt.xlabel("y_valid")
plt.show()

## Fit Bayesian linear regression model

Here we first define a [Bayesian linear regression](https://en.wikipedia.org/wiki/Bayesian_linear_regression) model with normal priors on the parameters.
Then we train it on the training data we prepared above.

In [ ]:
def bayesian_linear_regression_model(x, y=None):
    """Bayesian linear regression with normal priors."""
    # priors
    n_features = x.shape[1]
    w = pyro.sample("weight", pyro.distributions.Normal(0., 1.).expand([n_features]).to_event(1))
    b = pyro.sample("bias", pyro.distributions.Normal(0., 1.))
    sigma = pyro.sample("sigma", pyro.distributions.HalfNormal(1.))
    # likelihood
    mu = (x @ w + b)
    with pyro.plate("data", len(x)):
        pyro.sample("obs", pyro.distributions.Normal(mu, sigma), obs=y)
    return mu

In [ ]:
# train the model
def train(
    model,
    x_train,
    y_train,
    x_valid,
    y_valid,
    n_steps=1000,
    eval_freq=100,
):
    pyro.clear_param_store()
    guide = pyro.infer.autoguide.AutoDiagonalNormal(model)
    optimiser = pyro.optim.Adam({"lr": 0.01})
    svi = pyro.infer.SVI(model, guide, optimiser, loss=pyro.infer.Trace_ELBO())
    errors = defaultdict(list)
    for step in range(n_steps):
        elbo = svi.step(x_train, y_train)
        if step == 0 or (step + 1) % eval_freq == 0:
            train_loss = svi.evaluate_loss(x_train, y_train) / len(x_train)
            valid_loss = svi.evaluate_loss(x_valid, y_valid) / len(x_valid)
            errors["train_step"].append(step + 1)
            errors["train_loss"].append(train_loss)
            errors["valid_loss"].append(valid_loss)
            print(f"[{step + 1:5d}] train loss: {train_loss:7.4f}, valid loss: {valid_loss:7.4f}")
    return guide, errors

guide, errors = train(bayesian_linear_regression_model, x_train, y_train, x_valid, y_valid)

In [ ]:
# plot training curve
plt.figure()
plt.plot(errors["train_step"], errors["train_loss"], label="train loss")
plt.plot(errors["train_step"], errors["valid_loss"], label="valid loss")
plt.xlabel("training step"); plt.ylabel("loss")
plt.legend()
plt.grid()
plt.show()

## Sample posterior distribution

Now that we have a trained model, we can draw samples of the trained model parameters and predictions from the posterior distribution so we can analyse them below. 

In [ ]:
def sample_posterior(model, guide, x, n_samples=1000):
    predictive = pyro.infer.Predictive(model, guide=guide, num_samples=n_samples, return_sites=("weight", "bias", "sigma", "obs", "_RETURN",))
    raw_samples = predictive(x)
    samples = {"raw_samples": raw_samples}
    for i in range(len(features)):
        samples[f"weight{i}"] = raw_samples["weight"][:,:,i].squeeze()
    samples["bias"] = raw_samples["bias"].squeeze()#.numpy()
    samples["sigma"] = raw_samples["sigma"].squeeze()#.numpy()
    samples["mu"] = raw_samples["_RETURN"].squeeze()#.numpy()
    return samples

samples_train = sample_posterior(bayesian_linear_regression_model, guide, x_train)
samples_valid = sample_posterior(bayesian_linear_regression_model, guide, x_valid)

## Posterior predictive distribution

First, we check the predictions on the training and validation data to see how well the model fits the data.

In [ ]:
def r2(y_true, y_pred):
    """Compute coefficient of determination."""
    ssr = torch.sum((y_true - y_pred)**2)
    sst = torch.sum((y_true - torch.mean(y_true))**2)
    return 1 - (ssr / sst)

def mae(y_true, y_pred):
    """Compute mean absolute error."""
    return torch.mean(torch.abs(y_true - y_pred))

def evaluate_predictions(y_true, y_pred, lim=(-3,3), figsize=(8,4)):
    _r2 = r2(y_true, y_pred)  # coefficient of determination
    _mae = mae(y_true, y_pred)  # mean absolute error
    print(f"r2: {_r2:.4f}, mae: {_mae:.4f}\n")
    # plot
    plt.figure(figsize=figsize)
    plt.subplot(121)
    plt.plot(lim, lim, color="k", linestyle="--", linewidth=1)
    plt.plot(y_true.numpy(), y_pred.numpy(), ".", alpha=.1)
    plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("y_true"); plt.ylabel("y_pred")
    plt.grid()
    plt.subplot(122)
    plt.hist(y_true.numpy() - y_pred.numpy(), bins=20)
    plt.xlim(lim)
    plt.xlabel("y_true - y_pred")
    plt.tight_layout()
    plt.show()

In [ ]:
# evaluate on training data
evaluate_predictions(y_train, samples_train["mu"].mean(axis=0).detach())

In [ ]:
# evaluate on validation data
evaluate_predictions(y_valid, samples_valid["mu"].mean(axis=0))

On the log transformed `SEI_thickness(m)` output, the linear model performs rather well.
But there is still room for improvement.

## Plot posterior distribution

Below we first plot the distributions of the model parameters.
Parameters that are close to zero have a small effect on the output.

We can actually compute the effects by multiplying the (mean) parameters with the corresponding input data and plot the resulting distributions. 
As expected, the close-to-zero parameters on average have very little effect in the output.

Finally we can quantify the feature importances by computing the absolute [t-statistic](https://en.wikipedia.org/wiki/T-statistic) of each parameter distribution:

$$
t_d = \frac{\text{mean}(w_d)}{\text{std}(w_d)}
$$

where $d$ denotes the input dimension.

If you want to know more about these methods, this chapter on linear regression from the Interpretable Machine Learning book explains them well: https://christophm.github.io/interpretable-ml-book/limo.html

In [ ]:
def evaluate_posterior(samples, x, features):
    # prepare parameter table
    posterior_df = pd.DataFrame.from_dict({k:v.numpy() for k,v in samples.items() if k not in ["raw_samples", "mu"]})
    
    # parameter box plot
    _ = posterior_df.boxplot(figsize=(posterior_df.shape[1],4), rot=90)
    plt.title("Posterior distribution of model parameters")
    plt.show()
    
    # effects box plot (w*x)
    effects = (samples["raw_samples"]["weight"].mean(axis=0) * x)
    effects_df = pd.DataFrame(effects.numpy(), columns=[f"x{i}: {f}" for i,f in enumerate(features)])
    _ = effects_df.boxplot(figsize=(effects_df.shape[1], 4), rot=90)
    plt.title("Effects")
    plt.show()
    
    # feature importance computed as absolute t-statistic |t|
    feature_importance = (posterior_df.mean() / posterior_df.std())[[c for c in posterior_df.columns if "weight" in c]].abs()
    plt.figure(figsize=(6,3))
    plt.bar(range(len(feature_importance)), feature_importance.values)
    plt.xticks(range(len(features)), [f"x{i}: {f}" for i,f in enumerate(features)], rotation=90)
    plt.xlabel("Feature"); plt.ylabel("feature importance")
    plt.show()

In [ ]:
# evaluate on validation data
evaluate_posterior(samples_valid, x_valid, features)

From this analysis, it looks like there are a few important inputs with a large effect on the output, which can already be really useful information. 
These insights can help us simplify the model by ignoring some of the least important inputs and focus our efforts on the more important inputs.

However, there are some important assumptions to keep in mind. 
* The validity of these results of course depends on how well the linear model fits the data.
* In this example we made the analysis with regards to the log transformed output and care should be taken if we were to back-transform the results to the original scale since this is a nonlinear transformation and the predictive distribution would no longer be Gaussian.
* Since this is a linear model, it does not reveal if there are any nonlinear effects and thus if the important inputs have the same effect along their entire range of variation or not.
Perhaps a nonlinear model could provide some additional insights? We will look at that in the next steps of the tutorial.